In [100]:
%load_ext autoreload
%autoreload 2

import ast
from pathlib import Path
from pprint import pprint
from dataclasses import asdict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
from docstring_format import *
from docstring_format.base import *
from docstring_format.utils import *
import json

In [121]:
file_path = './tests/dummy_tests_functions.py'
file = Path(file_path)
raw_text = file.read_text()
lines = raw_text.splitlines()

In [122]:
results = {}

In [123]:
functions = get_functions(raw_text)
results['get_functions'] = [func.name for func in functions]

In [124]:
docstrings = [Docstring.from_ast(func, lines) for func in functions]

results['get_docstring_start_and_length'] = \
    [(docstring.start, docstring.length)
     for docstring in docstrings]

In [125]:
sections = [parse_sections(func, lines) for func in functions]
results['parse_sections'] = [asdict(section) for item in sections for section in item]

In [126]:
# remove function attribute from test
results['docstrings'] = [{k: v for k, v in asdict(docstring).items() if k != 'function'}
                         for docstring in docstrings]

In [127]:
results['sections_cleaned'] = [section.cleaned for item in sections for section in item]

In [130]:
results['docstrings_cleaned'] = [docstring.cleaned for docstring in docstrings]

In [136]:
script = ScriptFile(file_path)
results['script_cleaned']= script.cleaned

# write results

In [137]:
with open('./tests/dummy_test_functions_results.json', mode='w') as f:
    json.dump(results, f, indent=4, default=str)

# load from json

In [97]:
with open('./tests/dummy_test_functions_results.json', mode='r') as f:
    results = json.load(f)

section_map = {str(item): item for item in SectionType}

for item in results['parse_sections']:
    item['type'] = section_map[item['type']]
    
for item in results['docstrings']:
    for section in item['sections']:
        section['type'] = section_map[section['type']]